# M2177.003100 Deep Learning <br> Assignment #3 Part 3: Transformer

Copyright (C) Data Science Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. Written by Jeonghee Jo, October 2019, and modified by Jungbeom Lee, October 2020.

This is about Transformer (Vaswani et al., 2017).
https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf

Original blog post & code:
https://github.com/Kyubyong/transformer (Tensorflow)
https://pytorch.org/tutorials/beginner/transformer_tutorial.html (PyTorch)


That said, you are allowed to copy paste the codes from the original repo.
HOWEVER, <font color=red> try to implement the model yourself first </font>, and consider the original source code as a last resort.
You will learn a lot while wrapping around your head during the implementation. And you will understand nuts and bolts of Transformers more clearly in a code level.

### Submitting your work:
<font color=red>**DO NOT clear the final outputs**</font> so that TAs can grade both your code and results.  
Once you have done **all Assignment Part 1-3**, run the *CollectSubmission.sh* script with your **Student number** as input argument. <br>
This will produce a zipped file called *[Your student number].zip*. Please submit this file on ETL. &nbsp;&nbsp; (Usage: ./*CollectSubmission.sh* 20xx-xxxxx)





### The Grading is as follows:

This assignment is an on/off one: just make this notebook **"work"** without problem by: 

1. Train your model using at least <font color=red> 12 different hyperparameter set </font>. Report performance results computed in the last code block <font color=red> for corresponding each hyperparameter set </font>. Plus, <font color=red> submit the one checkpoint file </font> of your best model. 

2. Please provide the analysis of changed hyper-parameters. (10 points)

The details are described in <font color=red>**transformer_modules.py**</font>. (There is nothing to implement in this notebook.)


Now proceed to the code.


### Hyper-parameters

In [1]:
# change hyper-parameters in this code block!
import random

emsize = random.randint(200,600) # embedding dimension
nhid = random.randint(200,600) # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = random.randint(2,7) # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = random.randint(2,7) # the number of heads in the multiheadattention models
emsize = int(emsize/nhead)*nhead # for dividable
dropout = random.uniform(0.1,0.5) # the dropout value
batch_size = 128
eval_batch_size = 128
epochs = 3 # The number of epochs

# For Best Model
emsize = 416
nhead = 2
nhid = 399
nlayers = 2
dropout = 0.11775754439467333

### Load and Batch Data

In [2]:
# pip install torchtext==0.6.0
import torch
import torchtext
from torchtext.data.utils import get_tokenizer
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
%env CUDA_VISIBLE_DEVICES = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)



env: CUDA_VISIBLE_DEVICES=0


In [3]:
from transformer_modules import *
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [4]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [5]:
best_val_loss = float("inf")

best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()
torch.save(best_model.state_dict(), 'models/Transformer.pth')

/home/jaekyungcho/anaconda3/envs/deep-learning-20/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:350: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |   200/  465 batches | lr 5.00 | ms/batch 76.70 | loss  8.48 | ppl  4826.58
| epoch   1 |   400/  465 batches | lr 5.00 | ms/batch 75.47 | loss  6.73 | ppl   839.00
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 36.50s | valid loss  6.10 | valid ppl   444.33
-----------------------------------------------------------------------------------------
| epoch   2 |   200/  465 batches | lr 4.51 | ms/batch 76.22 | loss  6.15 | ppl   468.81
| epoch   2 |   400/  465 batches | lr 4.51 | ms/batch 76.11 | loss  5.93 | ppl   374.74
-----------------------------------------------------------------------------------------
| end of epoch   2 | time: 36.58s | valid loss  5.70 | valid ppl   297.63
-----------------------------------------------------------------------------------------
| epoch   3 |   200/  465 batches | lr 4.29 | ms/batch 76.64 | loss  5.71 | ppl   301.15
| epoch   3 |   400/  465 batches | lr 4.29 | m

In [6]:
best_model_loaded = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
best_model_loaded.load_state_dict(torch.load('models/Transformer.pth'), strict=True)
test_loss = evaluate(best_model_loaded, test_data)
print('=' * 89)
print('')
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}  '.format(
    test_loss, math.exp(test_loss)))
print('| Hyperparam Set | emsize {} | nhead {} | nhid {} | nlayers {} | dropout {}  \n'.format(
    emsize, nhead, nhid, nlayers, dropout))
print('=' * 89)


| End of training | test loss  5.42 | test ppl   225.02  
| Hyperparam Set | emsize 416 | nhead 2 | nhid 399 | nlayers 2 | dropout 0.11775754439467333  



# Please provide your analysis on each hyper-parameter.

총 12개의 hyperparameter set에 대하여 test loss 와 test ppl을 구하여 나타내었다.

hyperparameter의 경우 random함수를 사용하여 sampling 하였다. emsize가 nhead로 나누어 떨어져야하기 때문에 그 부분을 조금 조정한 것 이외에는 random 함수에 의존하여 sampling 하였다. sampling 범위의 경우 상식적인 선에서 임의로 결정하였다.

그 결과는 아래와 같다.

========Set 1===========================================================================

| End of training | test loss  5.55 | test ppl   257.36  
| Hyperparam Set | emsize 200 | nhead 2 | nhid 200 | nlayers 2 | dropout 0.2  

=========================================================================================

========Set 2============================================================================

| End of training | test loss  5.61 | test ppl   273.16  
| Hyperparam Set | emsize 400 | nhead 2 | nhid 436 | nlayers 4 | dropout 0.42629237581765866  

=========================================================================================

========Set 3============================================================================

| End of training | test loss  5.66 | test ppl   285.89  
| Hyperparam Set | emsize 484 | nhead 4 | nhid 337 | nlayers 4 | dropout 0.3585654657685182  

=========================================================================================

========Set 4============================================================================

| End of training | test loss  5.73 | test ppl   307.61  
| Hyperparam Set | emsize 412 | nhead 4 | nhid 404 | nlayers 3 | dropout 0.4923271460494234  

=========================================================================================

========Set 5============================================================================

| End of training | test loss  5.66 | test ppl   286.79  
| Hyperparam Set | emsize 478 | nhead 2 | nhid 435 | nlayers 2 | dropout 0.4453517332554101  

=========================================================================================

========Set 6============================================================================

| End of training | test loss  5.55 | test ppl   257.85  
| Hyperparam Set | emsize 280 | nhead 4 | nhid 527 | nlayers 3 | dropout 0.33495872352126155  

=========================================================================================

========Set 7============================================================================

| End of training | test loss  5.48 | test ppl   239.25  
| Hyperparam Set | emsize 362 | nhead 2 | nhid 497 | nlayers 3 | dropout 0.27982980144127834  

=========================================================================================

========Set 8============================================================================

| End of training | test loss  5.38 | test ppl   216.49  
| Hyperparam Set | emsize 416 | nhead 2 | nhid 399 | nlayers 2 | dropout 0.11775754439467333  

=========================================================================================

========Set 9============================================================================

| End of training | test loss  6.67 | test ppl   788.14  
| Hyperparam Set | emsize 588 | nhead 6 | nhid 287 | nlayers 7 | dropout 0.21496531402778696  

=========================================================================================

========Set 10===========================================================================

| End of training | test loss  5.42 | test ppl   225.32  
| Hyperparam Set | emsize 504 | nhead 7 | nhid 511 | nlayers 3 | dropout 0.1708669080824745  

=========================================================================================

========Set 11===========================================================================

| End of training | test loss  5.79 | test ppl   327.94  
| Hyperparam Set | emsize 376 | nhead 4 | nhid 516 | nlayers 5 | dropout 0.47521358575208905  

=========================================================================================

========Set 12===========================================================================

| End of training | test loss  5.74 | test ppl   312.20  
| Hyperparam Set | emsize 366 | nhead 2 | nhid 428 | nlayers 3 | dropout 0.4774593507973922  

=========================================================================================

간략하게 결과를 살펴보면, dropout 이 작을 수록 test loss가 작다는 것을 알 수 있다. 
이는 epoch가 충분히 길지 않기 때문에, dropout이 작을수록 초반 학습이 빨라 이러한 결과가 도출되었을 가능성이 있다.

또한 nlayers가 너무 크거나, nhid가 emsize에 비해 작을 때에도 좋지 않을 결과가 도출됨을 알 수 있었다.
이 또한 충분하지 않은 epoch로 인해 deep layer일 수록 초반 학습이 느려 이러한 결과가 도출되었을 수 있다.

최종적으로 가장 test loss 가 작은 hyperparameter set은 8번 set으로 
| Hyperparam Set | emsize 416 | nhead 2 | nhid 399 | nlayers 2 | dropout 0.11775754439467333 
의 hyperparameter set을 가진다. 적당히 큰 embedding size와 hidden dimension, 그리고 낮은 dropout, 상대적으로 얇은 네트워크 depth로 인해 초기 학습이 빠르게 진행되었다고 판단된다.